In [1]:
# This branch of qiskit is required:  pip install git+https://github.com/LNoorl/qiskit-terra.git@feature/sk-pass

import numpy as np
import dreamcoder as dc
from dreamcoder.domains.quantum_circuits.primitives import *
from dreamcoder.domains.quantum_circuits.tasks import *
import dill as pickle
import time
from tqdm import trange
import random
import bin.binutil
import datetime
dc.domains.quantum_circuits.primitives.GLOBAL_LIMITED_CONNECTIVITY = False
%load_ext line_profiler
%load_ext autoreload
%autoreload 2


In [2]:
from dreamcoder.frontier import Frontier, FrontierEntry
from dreamcoder.fragmentGrammar import FragmentGrammar
from dreamcoder.grammar import Grammar
from dreamcoder.program import Program
from dreamcoder.program import Abstraction


In [3]:
n_qubit_tasks = 3
evaluationTimeout = 0.01
settings = {
    "structurePenalty": 6,  # increase regularization 3 4 (it awas 1) look at a few [1,15]
    "pseudoCounts": 10,  # increase 100 test a few values
    "a": 3,
    "topK": 2,
}

In [11]:
def visualize_program(code:str):
    "Helper function to plot a circuit program"
    arguments = (*range(n_qubit_tasks), (n_qubit_tasks, ()))
    program =  Program.parse(code)
    circuit = execute_program(program, arguments)
    reconstructed_circuit = get_qiskit_circuit(circuit)
    print(code)
    print(reconstructed_circuit.circuit)

In [12]:
primitives = [
    p_hadamard,
    p_t,
    p_tdg,
    p_cnot,
    p_x,
]
grammar = Grammar.uniform(primitives, continuationType=tcircuit)

In [13]:
programs = [
    "(lambda (lambda (lambda (lambda (cnot (x $0 $1) $1 $2)))))",
    "(lambda (lambda (lambda (lambda (cnot (x (x (x $0 $1) $1) $1) $1 $2)))))",
    "(lambda (lambda (lambda (lambda (cnot (x (h (x $0 $2) $2) $2) $2 $1)))))",
    "(lambda (lambda (lambda (lambda (cnot (x (h (x (h $0 $2) $2) $2) $2) $2 $1)))))",
    "(lambda (lambda (lambda (lambda (cnot (x (h (x (h (x (h (x $0 $2) $2) $2) $2) $2) $2) $2) $2 $1)))))",
    
]

In [14]:
visualize_program(programs[1])

(lambda (lambda (lambda (lambda (cnot (x (x (x $0 $1) $1) $1) $1 $2)))))
     ┌───┐┌───┐┌───┐     
q_0: ┤ X ├┤ X ├┤ X ├──■──
     └───┘└───┘└───┘┌─┴─┐
q_1: ───────────────┤ X ├
                    └───┘
q_2: ────────────────────
                         


In [15]:
tasks = {}  # dict of task:solution

for code in programs:
    arguments = (*range(n_qubit_tasks), (n_qubit_tasks, ()))
    program = Program.parse(code)
    circuit = execute_program(program, arguments)
    task = QuantumTask(f"t_{len(tasks):03d}_{code}", circuit)

    # 0 = solved, -inf = not solved
    likelihood = task.logLikelihood(program, evaluationTimeout)
    prior = grammar.logLikelihood(task.request, program)

    frontier_entry = FrontierEntry(
        program=program, logLikelihood=likelihood, logPrior=prior
    )

    tasks[task] = Frontier(
        frontier=[frontier_entry],  # multiple solutions are allowed
        task=task,
    )

In [16]:
frontiers = [f for f in tasks.values()]

new_grammar, new_frontiers = FragmentGrammar.induceFromFrontiers(
    g0 =grammar,
    frontiers = frontiers,
    **settings
)

new_grammar, new_frontiers

fragmentGrammar.py:298 > Inducing a grammar from 5 frontiers
fragmentGrammar.py:327 > Starting score -111.17389390053093
fragmentGrammar.py:334 > Proposed 825 fragments.
fragmentGrammar.py:365 > New primitive of type tcircuit	(x (h $0 $1) $1)	
(score = -111.094551; dScore = 0.079343; <uses> = 5.802837)
fragmentGrammar.py:378 > 	(<uses> in rewritten frontiers: 6.000000)
fragmentGrammar.py:334 > Proposed 81 fragments.
fragmentGrammar.py:397 > Old joint = -80.930409	New joint = -63.053194

fragmentGrammar.py:417 > 0.000000 / 22.000000	h
fragmentGrammar.py:417 > 0.000000 / 22.000000	t
fragmentGrammar.py:417 > 0.000000 / 22.000000	tdg
fragmentGrammar.py:417 > 5.000000 / 22.000000	cnot
fragmentGrammar.py:417 > 6.000000 / 22.000000	x
fragmentGrammar.py:417 > 6.000000 / 22.000000	#(lambda (lambda (x (h $0 $1) $1)))


(<dreamcoder.grammar.Grammar at 0x7fe42ad45250>,
 [Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (cnot (x $0 $1) $1 $2))))), logPrior=-8.443458133549708, logLikelihood=0.0], task=t_000_(lambda (lambda (lambda (lambda (cnot (x $0 $1) $1 $2)))))),
  Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (cnot (x (x (x $0 $1) $1) $1) $1 $2))))), logPrior=-14.355792502774959, logLikelihood=0.0], task=t_001_(lambda (lambda (lambda (lambda (cnot (x (x (x $0 $1) $1) $1) $1 $2)))))),
  Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (cnot (#(lambda (lambda (x (h $0 $1) $1))) $2 (x $0 $2)) $2 $1))))), logPrior=-11.433719529505309, logLikelihood=0.0], task=t_002_(lambda (lambda (lambda (lambda (cnot (x (h (x $0 $2) $2) $2) $2 $1)))))),
  Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (cnot (#(lambda (lambda (x (h $0 $1) $1))) $2 (#(lambda (lambda (x (h $0 $1) $1))) $2 $0)) $2 $1))))), logPrior=-11.4678137

In [15]:
type(new_grammar.primitives[-1])

dreamcoder.program.Invented

In [39]:
new_grammar.primitives[-1]

#(lambda (lambda (x (h $0 $1) $1)))

In [40]:
print(new_grammar)

-0.173272	t0	$_
-0.693147	tcircuit -> int -> tcircuit	x
-0.757686	tcircuit -> int -> int -> tcircuit	cnot
-1.163151	tcircuit -> int -> tcircuit	h
-1.163151	tcircuit -> int -> tcircuit	t
-1.163151	tcircuit -> int -> tcircuit	tdg
-0.693147	int -> tcircuit -> tcircuit	#(lambda (lambda (x (h $0 $1) $1)))


In [51]:
p = new_frontiers[0].entries[0].program

In [69]:
p.body.body.body.body

(cnot (x $0 $1) $1 $2)

### Defining a program

In [4]:
arguments = (*range(n_qubit_tasks), (n_qubit_tasks, ()))
program =  Program.parse(code)
circuit = execute_program(program, arguments)
reconstructed_circuit = get_qiskit_circuit(circuit)

NameError: name 'code' is not defined

In [19]:
# p =Program.parse("(lambda (lambda (lambda (lambda (cnot (#(lambda (lambda (x (h $0 $1) $1))) $2 (x $0 $2)) $2 $1)))))")
# p =Program.parse("cnot")
p = Program.parse("(lambda $1)")
p

(lambda $1)

In [23]:
p.evaluate([3])([])

3

In [57]:
p.body.body.body.body.f.f.x.f.f.body.body.body.f.x.x

$1

In [7]:
type(p)

dreamcoder.program.Abstraction

In [8]:
p.body.x.evaluate

<bound method Index.evaluate of $0>

In [9]:
ee = p.evaluate([1])
ee(no_op(2))(1)(0)

(2, (('cnot', 1, 0),))

In [10]:
import inspect
inspect.getsource(ee)

'        return lambda x: self.body.evaluate([x] + environment)\n'